In [ ]:
import time, os
#from dask import dataframe as dd
import pandas as pd
import multiprocessing
import numpy as np

In [ ]:
%%time
# Takes 2 minutes with Dask.

import dask.dataframe as dd

# This file is 1.4GB
# Created in 01_extract_outcomes.ipynb
orig_df = dd.read_csv("data/MIMIC_001_chartevents.csv",
                      dtype = {'ICUSTAY_ID': 'UInt64',
                               'VALUE': 'object',
                               'ERROR': 'UInt64',
                               'STOPPED': 'object',
                               'CGID': 'UInt64',
                               'WARNING': 'UInt64'})
#                      dtype = {'RESULTSTATUS': 'object',
#                             'STOPPED': 'object', 'VALUE': 'object'},
#                      assume_missing = True)
orig_df.compute()

In [ ]:
%%time

# Takes a minute or two.
len(orig_df[orig_df.Dialysis==1].HADM_ID.unique())

In [ ]:
%%time

# Takes a minute.
len(orig_df[orig_df.ITEMID==226118].HADM_ID.unique())

In [ ]:
#Finding outcomes of interest from D_ITEMS (dictionary)

#d_items_file='M:/Raw_CSV/D_ITEMS.csv'
d_items = pd.read_csv("data-raw/mimic/D_ITEMS.csv")


In [ ]:
d_items.head(5)

In [ ]:
#Getting all the items under Category = Dialysis
dialysis_items=d_items[d_items.CATEGORY=='Dialysis'].ITEMID.unique().tolist()
print('by category: ', len(dialysis_items))

additional_hd_items=[226118, 227357, 225725, 226499, 224154, 225810, 227639, 225183, 227438, 224191, 225806, 
           225807, 228004, 228005, 228006, 224144, 224145, 224149, 224150, 224151, 224152, 224153, 
           224404, 224406, 226457, 225959, 224135, 224139, 224146, 225323, 225740, 225776, 225951, 
           225952, 225953, 225954, 225956, 225958, 225961, 225963, 225965, 225976, 225977, 227124, 
           227290, 227638, 227640, 227753, 40788, 40907, 41063, 41147, 41307, 41460, 41620, 41711, 
           41791, 41792, 42562, 43829, 44037, 44188, 44526, 44527, 44584, 44591, 44698, 44927, 44954, 
           45157, 45268, 45352, 45353, 46012, 46013, 46172, 
           46173, 46250, 46262, 46292, 46293, 46311, 46389, 46574, 46681, 46720, 46769, 46773, 
            40386, 40425, 40426, 40507, 40613, 40624, 40690, 40745, 40789, 40881, 40910, 41016, 
            41034, 41069, 41112, 41250, 41374, 41417, 41500, 41527, 41623, 41635, 41713, 
            41750, 41829, 41842, 41897, 42289, 42388, 42464, 42524, 42536, 42868, 42928, 
            42972, 43016, 43052, 43098, 43115, 43687, 43941, 44027, 44085, 44193, 44199, 
            44216, 44286, 44567, 44843, 44845, 44857, 44901, 44943, 45479, 45828, 46230, 
            46232, 46394, 46464, 46712, 46713, 46715, 46741, 
            225441, 225802, 225803, 225805, 224270, 225809, 225955, 225436, 
                     225318, 225319, 225321, 225322, 225324]
dialysis_items.extend(additional_hd_items)
print('adding additional items:', len(dialysis_items))

#Now brute forcing the ITEMIDs 
def find_hd_item(string): 
    items=['dialy', 'dialysis', 'tunneled', 'crrt', 'cvvh', 'scuf']
    for i in items: 
        if i in string.lower(): return True 
d_items['Dialysis']=d_items.LABEL.map(lambda x: 1 if find_hd_item(str(x)) else 0)
for i in d_items[d_items.Dialysis==1].ITEMID.unique().tolist(): 
    if i not in dialysis_items: dialysis_items.append(i)
print('final count with word search:', len(dialysis_items))

In [ ]:
#Prior dialysis item code 
#dialysis ITEMIDs: https://github.com/MIT-LCP/mimic-code/blob/master/concepts/rrt.sql
dialysis_items2=d_items[d_items.CATEGORY=='Dialysis'].ITEMID.unique().tolist()
print(len(dialysis_items2))

d_items.LABEL=d_items.LABEL.map(lambda x: str(x).lower())
def find_item(string): 
    items=['dialy', 'dialysis', 'tunneled', 'crrt', 'cvvh', 'scuf']
    for i in items: 
        if i in string: return True 
d_items['Dialysis']=d_items.LABEL.map(lambda x: 1 if find_item(x) else 0)
for i in d_items[d_items.Dialysis==1].ITEMID.unique().tolist(): 
    if i not in dialysis_items2: dialysis_items2.append(i)
print(len(dialysis_items2))

In [ ]:
#Function to find outcomes of interests only in CHARTEVENTS 
def find_item_id(integer, item_list): 
    if integer in item_list: return 1
    else: return 0

In [ ]:
%%time

#input_events='M:/Raw_CSV/INPUTEVENTS_CV.csv'

# 2.3GB, takes ~2.3 minutes.
input_df = dd.read_csv("data-raw/mimic/INPUTEVENTS_CV.csv",
                       dtype={'ORIGINALRATEUOM': 'object',
                              'RATEUOM': 'object',
                              'ORIGINALSITE': 'object',
                              'STOPPED': 'object'},
                       assume_missing = True)
input_df = input_df.compute()
print(input_df.shape)
input_df.head(5)

In [ ]:
%%time

# Slow - see if this can be sped up.
input_df['Dialysis']=input_df.ITEMID.map(lambda x: find_item_id(int(x), dialysis_items))

In [ ]:
input_final=input_df[input_df.Dialysis==1]
print(input_final.shape)

In [ ]:
#input_final.to_csv('M:/MIMIC_001_inputevents_hd.csv')
input_final.to_csv('data/MIMIC_001_inputevents_hd.csv')

In [ ]:
%%time

#chart_events='M:/Raw_CSV/CHARTEVENTS.csv'

# This takes a long time, file is 33GB.
# 6.75 mins on SSD.
df = dd.read_csv('data-raw/mimic/CHARTEVENTS.csv',
                 usecols = ['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID'],
                 # CK: do we really want this line below? consider removing.
                 assume_missing = True)
df = df.compute() # for dask version
df.head(5)

In [ ]:
%%time

# Very slow - try to speed up.
df['Dialysis']=df.ITEMID.map(lambda x: find_item_id(int(x), dialysis_items))

In [ ]:
chart_final=df[df.Dialysis==1]
print(chart_final.shape)
#chart_final.to_csv('M:/MIMIC_001_chartevents_hd.csv')
chart_final.to_csv('data/MIMIC_001_chartevents_hd.csv')

In [ ]:
%%time
# out_events='M:/Raw_CSV/OUTPUTEVENTS.csv'
out_df = pd.read_csv("data-raw/mimic/OUTPUTEVENTS.csv")
out_df.head(5)

In [ ]:

out_df['Dialysis']=out_df.ITEMID.map(lambda x: find_item_id(int(x), dialysis_items))

In [ ]:
out_final=out_df[out_df.Dialysis==1]
print(out_final.shape)
#out_final.to_csv('M:/MIMIC_001_outputevents_hd.csv')
out_final.to_csv('data/MIMIC_001_outputevents_hd.csv')

In [ ]:
%%time

# proc_events='M:/Raw_CSV/PROCEDUREEVENTS_MV.csv'
proc_df = pd.read_csv("data-raw/mimic/PROCEDUREEVENTS_MV.csv")
proc_df.head(5)

In [ ]:
proc_df['Dialysis']=proc_df.ITEMID.map(lambda x: find_item_id(int(x), dialysis_items))

In [ ]:
proc_final=proc_df[proc_df.Dialysis==1]
print(proc_final.shape)
#proc_final.to_csv('M:/MIMIC_001_procevents_hd.csv')
proc_final.to_csv('data/MIMIC_001_procevents_hd.csv')

In [ ]:
%%time

# dat_events='M:/Raw_CSV/DATETIMEEVENTS.csv'

dat_df = pd.read_csv("data-raw/mimic/DATETIMEEVENTS.csv")

dat_df.head(5)

In [ ]:
dat_df['Dialysis']=dat_df.ITEMID.map(lambda x: find_item_id(int(x), dialysis_items))

In [ ]:
dat_final=dat_df[dat_df.Dialysis==1]
print(dat_final.shape)
#dat_final.to_csv('M:/MIMIC_001_dataevents_hd.csv')
dat_final.to_csv('data/MIMIC_001_dataevents_hd.csv')

In [ ]:
len(input_final.HADM_ID.unique())

CK: these last two chunks appear to be old and the dataframe is not being used, so it seems to be ok that it's generating an error.

In [ ]:
df_final=df[(df.Mech_Vent==1) | (df.Dialysis==1) |(df.Trach==1)]

In [ ]:
df_final.shape

In [ ]:
df.columns